In [ ]:
import os
import time

import numpy as np
import torch
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from data_prep.SlicesDataset import SlicesDataset
from utils.utils import log_to_tensorboard
from utils.volume_stats import Dice3d, Jaccard3d
from networks.RecursiveUNet import UNet
from inference.UNetInferenceAgent import UNetInferenceAgent

import os
from os import listdir
from os.path import isfile, join

import numpy as np
from medpy.io import load
# from utils.utils import med_reshape

In [ ]:
class SlicesDataset(Dataset):
    """
    This class represents an indexable Torch dataset
    which could be consumed by the PyTorch DataLoader class
    """
    def __init__(self, data):
        self.data = data

        self.slices = []

        for i, d in enumerate(data):
            for j in range(d["image"].shape[0]):
                self.slices.append((i, j))

    def __getitem__(self, idx):
        """
        This method is called by PyTorch DataLoader class to return a sample with id idx

        Arguments: 
            idx {int} -- id of sample

        Returns:
            Dictionary of 2 Torch Tensors of dimensions [1, W, H]
        """
        slc = self.slices[idx]
        sample = dict()
        sample["id"] = idx

        # You could implement caching strategy here if dataset is too large to fit
        # in memory entirely
        # Also this would be the place to call transforms if data augmentation is used
        
        # TASK: Create two new keys in the "sample" dictionary, named "image" and "seg"
        # The values are 3D Torch Tensors with image and label data respectively. 
        # First dimension is size 1, and last two hold the voxel data from the respective
        # slices. Write code that stores the 2D slice data in the last 2 dimensions of the 3D Tensors. 
        # Your tensor needs to be of shape [1, patch_size, patch_size]
        # Don't forget that you need to put a Torch Tensor into your dictionary element's value
        # Hint: your 3D data sits in self.data variable, the id of the 3D volume from data array
        # and the slice number are in the slc variable. 
        # Hint2: You can use None notation like so: arr[None, :] to add size-1 
        # dimension to a Numpy array
        # <YOUR CODE GOES HERE>
        sample["image"] = []
        sample["seg"] = []
        

        return sample

    def __len__(self):
        """
        This method is called by PyTorch DataLoader class to return number of samples in the dataset

        Returns:
            int
        """
        return len(self.slices)

In [ ]:
train_loader = DataLoader(SlicesDataset(dataset[split["train"]]),

In [ ]:
root_dir = "/home/phenomx/dante/udacity/Applying_AI_to_3D_Medical_Imaging_Data/test_sections/nd320-c3-3d-imaging-starter/data/TrainingSet"
image_dir = os.path.join(root_dir, 'images')
label_dir = os.path.join(root_dir, 'labels')

images = [f for f in listdir(image_dir) if (isfile(join(image_dir, f)) and f[0] != ".")]
images[0]

In [ ]:
def med_reshape(image, new_shape):
    """
    This function reshapes 3D data to new dimension padding with zeros
    and leaving the content in the top-left corner

    Arguments:
        image {array} -- 3D array of pixel data
        new_shape {3-tuple} -- expected output shape

    Returns:
        3D array of desired shape, padded with zeroes
    """

    reshaped_image = np.zeros(new_shape)

    # TASK: write your original image into the reshaped image
    # <CODE GOES HERE>
    print(reshaped_image.shape)
    print(image.shape)
    # return reshaped_image

In [ ]:
f = images[0]
image, _ = load(os.path.join(image_dir, f))
label, _ = load(os.path.join(label_dir, f))

# TASK: normalize all images (but not labels) so that values are in [0..1] range
# <YOUR CODE GOES HERE>
slice_num = len(image.swapaxes(0,1))
for slice_ix in range(slice_num):
    image[:,slice_ix,:] = image[:,slice_ix,:].astype(np.single)/np.max(image[:,slice_ix,:])

# image = med_reshape(image, new_shape=(image.shape[0], image.shape[2], image.shape[1]))
# label = med_reshape(label, new_shape=(label.shape[0], label.shape[2], label.shape[1])).astype(int)

In [ ]:
med_reshape(image, new_shape=(image.shape[0], image.shape[2], image.shape[1]))
# label = med_reshape(label, new_shape=(label.shape[0], label.shape[2], label.shape[1])).astype(int)

In [153]:
import os
import json

from experiments.UNetExperiment import UNetExperiment
from data_prep.HippocampusDatasetLoader import LoadHippocampusData

import os
import time

import numpy as np
import torch
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from data_prep.SlicesDataset import SlicesDataset
from utils.utils import log_to_tensorboard
from utils.volume_stats import Dice3d, Jaccard3d
from networks.RecursiveUNet import UNet
from inference.UNetInferenceAgent import UNetInferenceAgent
from medpy.io import load

In [174]:
a, _ = load('/home/phenomx/dante/udacity/Applying_AI_to_3D_Medical_Imaging_Data/test_sections/nd320-c3-3d-imaging-starter/data/TrainingSet/images/hippocampus_010.nii.gz')
a.shape

(512, 512, 241)

In [154]:
root_dir = "/home/phenomx/dante/udacity/Applying_AI_to_3D_Medical_Imaging_Data/test_sections/nd320-c3-3d-imaging-starter/data/TrainingSet"
batch_size = 8
patch_size = 64

In [155]:
data = LoadHippocampusData(root_dir, y_shape = patch_size, z_shape = patch_size)

Processed 262 files, total 10222 slices


In [156]:
keys = range(len(data))

split = dict()


split["train"]=keys[:int(len(keys)*0.8)]
split["test"]=keys[int(len(keys)*0.8):int(len(keys)*0.9)]
split["val"]=keys[int(len(keys)*0.9):]

In [157]:
train_loader = DataLoader(SlicesDataset(data[split["train"]]),
                batch_size=batch_size, shuffle=True, num_workers=0)

In [158]:
# for i, batch in enumerate(train_loader):
#     print(i)
#     print(batch)
#     print(batch['id'])

In [159]:
data = data[split["train"]]

slices = []

for i, d in enumerate(data):
    for j in range(d["image"].shape[0]):
        slices.append((i, j))


In [168]:
data[0]["image"].shape

(34, 34, 50)

In [162]:
np.unique(data[0]["image"][0])

array([0.])

In [148]:
idx = 0
slc = slices[idx]
sample = dict()
sample["id"] = idx

In [149]:
slc

(0, 0)

In [152]:
a = np.unique(data[0]['image'][0,:,:][None,:,:])

In [151]:
a

array([0.])